In [2]:
import requests
from lxml import etree
import random
import time
import re
import csv
import threading
from queue import Queue
from requests.packages import urllib3
urllib3.disable_warnings()

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.92 Safari/537.36',
    'Referer': 'https://www.lagou.com/jobs/list_python?city=%E5%85%A8%E5%9B%BD&cl=false&fromSearch=true&labelWords=&suginput=&isSchoolJob=1',
    'Cookie': '_ga=GA1.2.1601189612.1535839741; user_trace_token=20180902060901-a1ab575c-ae33-11e8-81c5-525400f775ce; LGUID=20180902060901-a1ab5ecf-ae33-11e8-81c5-525400f775ce; index_location_city=%E5%85%A8%E5%9B%BD; JSESSIONID=ABAAABAABEEAAJADB2E073EDD93EFBF840ADE1477763A6D; _gid=GA1.2.739288378.1536963418; X_HTTP_TOKEN=56b4b5e3104da2589f89590e533ba25d; Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1536856454,1536963418,1537045319,1537049872; Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1537049872; LGSID=20180916061751-2fb51e87-b935-11e8-9c3a-525400f775ce; PRE_UTM=; PRE_HOST=localhost; PRE_SITE=http%3A%2F%2Flocalhost%3A8888%2Fnotebooks%2FPython%25E7%2588%25AC%25E8%2599%25AB%2F20.%25E5%25AE%259E%25E6%2588%2598-selenium%25E5%25AE%258C%25E7%25BE%258E%25E5%25AE%259E%25E7%258E%25B0%25E6%258B%2589%25E5%258B%25BE%25E7%25BD%2591%25E7%2588%25AC%25E8%2599%25AB.ipynb; PRE_LAND=https%3A%2F%2Fwww.lagou.com%2Fjobs%2F3117204.html; LGRID=20180916061751-2fb52056-b935-11e8-9c3a-525400f775ce; TG-TRACK-CODE=search_code; SEARCH_ID=5141aa4c795342538f5ba1f5e3622e88',
    'Host': 'www.lagou.com',
    'Origin': 'https://www.lagou.com',
    'X-Anit-Forge-Code': '0',
    'X-Anit-Forge-Token': 'None',
    'X-Requested-With': 'XMLHttpRequest'
}

URL = 'https://www.lagou.com/jobs/positionAjax.json?needAddtionalResult=false&isSchoolJob=1'

class LGSpider(threading.Thread):
    def __init__(self, page_queue, position_queue,*args, **kwargs):
        super(LGSpider, self).__init__(*args, **kwargs)
        self.page_queue = page_queue
        self.position_queue = position_queue
    
    def run(self):
        while True:
            try:
                if self.page_queue.empty():
                    break
                position_url = self.page_queue.get()
                response = requests.get(position_url, headers=HEADERS, verify=False)
                text = response.text
                html = etree.HTML(text)
                position_name = html.xpath("//span[@class='name']/text()")[0]
                company_name = html.xpath("//a//div//h2/text()")[0].strip()
                company_detail = html.xpath("//ul[@class='c_feature']//text()")
                company_domain = company_detail[2].strip()
                company_development = company_detail[7].strip()
                company_size = company_detail[12].strip()
                job_request_spans = html.xpath("//dd[@class='job_request']//span")
                salary = job_request_spans[0].xpath('.//text()')[0].strip()
                city = job_request_spans[1].xpath('.//text()')[0].strip()
                city = re.sub(r"[\s/]", "", city)
                work_years = job_request_spans[2].xpath('.//text()')[0].strip()
                work_years = re.sub(r"[\s/]", "", work_years)
                education = job_request_spans[3].xpath('.//text()')[0].strip()
                education = re.sub(r"[\s/]", "", education)
                address = "".join(html.xpath("//div[@class='work_addr']//text()")[:-2])
                address = re.sub(r"\s", "", address)
                description = "".join(html.xpath("//dd[@class='job_bt']//text()")).strip()
                job = {
                    'position_name': position_name,
                    'company_name': company_name,
                    'company_development': company_development,
                    'company_size': company_size,
                    'salary': salary,
                    'city': city,
                    'work_years': work_years,
                    'education': education,
                    'address': address,
                    'description': description,
                    'position_url': position_url
                }
                self.position_queue.put(job)
                time.sleep(random.random())
                
            except:
                print('Failed:', position_url)
                self.page_queue.put(position_url)
                print('Have added it back to the queue.')
                continue

class LGWriter(threading.Thread):
    def __init__(self, page_queue, position_queue, writer, gLock, file, *args, **kwargs):
        super(LGWriter, self).__init__(*args, **kwargs)
        self.page_queue = page_queue
        self.position_queue = position_queue
        self.writer = writer
        self.lock = gLock
        self.file = file
    
    def run(self):
        while True:
            self.lock.acquire()
            if self.page_queue.empty() and self.position_queue.empty():
                time.sleep(5)
                self.file.close()
                print('All Done!')
                break
            position_info = self.position_queue.get()
            self.writer.writerow(position_info)
            self.lock.release()
            print(position_info['position_url'] + ' is Done.')
            

def main():
    kd = 'python'
    page_queue = Queue(1000)
    position_queue = Queue(500)
    gLock = threading.Lock()

    file = open('./lagou_positions/' + 'lagou_%s2.csv' % kd, 'a', newline='', encoding='utf-8-sig')
    csv_headers = ['position_name', 'company_name', 'company_development', 'company_size', 'salary', 'city', 'work_years', 'education', 'address', 'description', 'position_url']
    writer = csv.DictWriter(file, csv_headers)
    writer.writeheader()
    
    for x in range(64, 1000):
        try:
            data = {
                'first': "false"
            }
            data['kd'] = kd
            data['pn'] = x
            response = requests.post(URL, headers=HEADERS, data=data, verify=False)
            result = response.json()
            positions = result['content']['positionResult']['result']
            for position in positions:
                positionId = position['positionId']
                position_url = 'https://www.lagou.com/jobs/%s.html' % positionId
                page_queue.put(position_url)
            print('Reading ajax from page:', str(x))
            time.sleep(random.random())
        except:
            print('Stop Here: page', str(x))
            break
        

    for x in range(1):
        t = LGSpider(page_queue, position_queue)
        t.start()
    for x in range(5):
        t = LGWriter(page_queue, position_queue, writer, gLock, file)
        t.start()

main()

Reading ajax from page: 64
Reading ajax from page: 65
Reading ajax from page: 66
Stop Here: page 67
https://www.lagou.com/jobs/3637306.html is Done.
https://www.lagou.com/jobs/4952030.html is Done.
https://www.lagou.com/jobs/5112000.html is Done.
https://www.lagou.com/jobs/4957843.html is Done.
https://www.lagou.com/jobs/4868641.html is Done.
https://www.lagou.com/jobs/5079723.html is Done.
https://www.lagou.com/jobs/3825799.html is Done.
https://www.lagou.com/jobs/4104709.html is Done.
https://www.lagou.com/jobs/4961223.html is Done.
https://www.lagou.com/jobs/5017555.html is Done.
https://www.lagou.com/jobs/3886748.html is Done.
https://www.lagou.com/jobs/5074240.html is Done.
https://www.lagou.com/jobs/5064637.html is Done.
https://www.lagou.com/jobs/5057188.html is Done.
https://www.lagou.com/jobs/4753941.html is Done.
https://www.lagou.com/jobs/5017552.html is Done.
https://www.lagou.com/jobs/4704648.html is Done.
https://www.lagou.com/jobs/5127796.html is Done.
https://www.lagou.